원문: Moler, Cleve B. Numerical computing with MATLAB. Society for Industrial and Applied Mathematics, 2008.

# 1.4 마방진

마방진으로 흥미로운 예제 행렬을 만들 수 있다.

In [ ]:
import numpy as np


def magic(N):
    if (N % 2):
        result = magic_odd(N)
    elif 0 == (N % 4):
        result = magic_four(N)
    elif (N % 4):
        result = magic_even_odd(N)
    else:
        raise NotImplementedError

    # check result
    assert_magic_square(result, N)

    return result


def magic_odd(N):
    # scipython.com/book/chapter-6-numpy/examples/creating-a-magic-square/
    magic_square = np.matrix(np.zeros((N, N), dtype=int))

    n = 1
    i, j = 0, N // 2

    while n <= N ** 2:
        magic_square[i, j] = n
        n += 1
        newi, newj = (i - 1) % N, (j + 1) % N
        if magic_square[newi, newj]:
            i += 1
        else:
            i, j = newi, newj

    return magic_square


def magic_four(N):
    # https://m.blog.naver.com/askmrkwon/220768685076 (in Korean)
    magic_ascending = np.array(range(1, N ** 2 + 1), dtype=int).reshape((N, N))
    magic_descending = np.array(range(N ** 2, 0, -1), dtype=int).reshape((N, N))
    magic_four_mat = np.matrix(np.zeros((N, N)), dtype=int)

    for i_row in range(0, N):
        for j_col in range(0, N):
            if (0 == (abs(i_row - j_col) % 4)) or (0 == (i_row + j_col + 1) % 4):
                magic_four_mat[i_row, j_col] = magic_descending[i_row, j_col]
            else:
                magic_four_mat[i_row, j_col] = magic_ascending[i_row, j_col]

    assert 0 < magic_four_mat.min()

    return magic_four_mat


def magic_even_odd(even):
    # https://m.blog.naver.com/askmrkwon/220768685076 (in Korean)

    assert 0 == (even % 2)

    odd = even // 2  # 2n + 1

    assert (odd % 2)

    upper_left = magic_odd(odd)
    lower_right = upper_left + (odd * odd)
    upper_right = lower_right + (odd * odd)
    lower_left = upper_right + (odd * odd)

    assert upper_left.min() == 1
    assert upper_left.max() == odd * odd

    assert lower_right.min() == upper_left.max() + 1
    assert lower_right.max() == odd * odd * 2

    assert upper_right.min() == lower_right.max() + 1
    assert upper_right.max() == odd * odd * 3

    assert lower_left.min() == upper_right.max() + 1
    assert lower_left.max() == even * even

    n = (odd - 1) // 2

    # exchange left
    temp = upper_left[:, 0:n].copy()
    upper_left[:, 0:n] = lower_left[:, 0:n]
    lower_left[:, 0:n] = temp

    # exchange right
    if 1 < n :
        temp = upper_right[:, -(n - 1):].copy()
        upper_right[:, -(n - 1):] = lower_right[:, -(n - 1):]
        lower_right[:, -(n - 1):] = temp

    # exchange left middle
    temp = upper_left[n, (n - 1):(n + 1)].copy()
    upper_left[n, (n - 1):(n + 1)] = lower_left[n, (n - 1):(n + 1)]
    lower_left[n, (n - 1):(n + 1)] = temp

    result = np.row_stack((np.column_stack((upper_left, upper_right)),
                           np.column_stack((lower_left, lower_right))))

    # check result
    assert_magic_square(result, even)

    return result


def assert_magic_square(mat, n):
    # check result
    magic_sum = np.sum(mat) / n
    row_sum_vector = np.sum(mat, 0)
    col_sum_vector = np.sum(mat, 1)
    assert np.abs(row_sum_vector - magic_sum).max() < 1e-7
    assert np.abs(row_sum_vector - magic_sum).min() < 1e-7
    assert np.abs(col_sum_vector - magic_sum).max() < 1e-7
    assert np.abs(col_sum_vector - magic_sum).min() < 1e-7


A = magic(3)
A

아래 명령은 모든 요소의 합을 계산한다.

In [ ]:
np.sum(A)

각 열의 합은 아래 명령으로 계산한다.

In [ ]:
np.sum(A, 0)

각 행의 합은 다음과 같이 구한다.

In [ ]:
np.sum(A, 1)

다음은 주대각선의 합을 계산한다.

In [ ]:
np.sum(np.diag(A))

반 대각선은 오른쪽 위에서 왼쪽 아래를 잇는 대각선으로, 선형 대수에서는 중요성이 덜하기 때문에, 합을 구하는 절차가 다소 복잡히다. 한가지 방법은 행렬을 위아래로 뒤집는 함수를 사용하는 것이다.

In [ ]:
np.sum(np.diag(np.flipud (A)))

따라서 모든 합이 같다는 것을 확인할 수 있다.

왜 합은 15인가? 아래 명령

In [ ]:
sum(np.arange(1, 9+1))

은 1 부터 9 까지의 정수의 합이 45임을 말한다. 이 정수들이 3 열에 나누어지고 합계가 같다면 그 합은

In [ ]:
sum(np.arange(1, 9+1)) / 3

15가 될 것이다.

OHP 에 투명지를 올려놓는 방법은 모두 8가지가 있다. 비슷하게, 3렬의 마방진을 배열하는 8가지 방법이 있다.  아래 명령은 각각의 방법을 모두 표시한다.

In [ ]:
for k in range(0, 4):
    print(np.rot90(A, k))
    print(np.rot90(A.T, k))

이제 선형 대수를 좀 살펴보자. 행렬식은 다음과 같다.

In [ ]:
import numpy.linalg as na
na.det(A)

역행렬은

In [ ]:
X = A.I
X

이다. 분수 형태로 표시할 수 있다면 더 좋다.

In [ ]:
import fractions
np.set_printoptions(formatter={'all': lambda x: str(fractions.Fraction(x).limit_denominator())})
print(X)

아래와 같이 하면 원상 복귀 될 것이다.

In [ ]:
np.set_printoptions(formatter=None)

전산 선형 대수에서 중요한 3가지 값으로 행렬의 놂, 고유치, 특이값 등이 있다.

In [ ]:
r = na.norm(A, np.inf)
e, w = na.eig(A)
u, s, v_h = na.svd(A)
print('r =')
print(r)
print('e =')
print(e)
print('s =')
print(s)

마방진의 한 열의 합은 위 3 값에서 모두 나타난다. 고유 벡터 (아래의 w), 좌 우 특이 벡터 (각각 아래의 u, v) 가운데 하나는 모두 1인 것이 있기 때문이다. (w, u, v 의 각 열 가운데 하나씩은 모든 항의 값이 같음)

In [ ]:
print('w =')
print(w)
print('u =')
print(u)
print('v =')
print(v_h.H)

지금까지 이 절의 모든 연산은 부동소숫점이었다. 대부분의 과학 기술 계산, 특히 큰 행렬의 연산은 이렇게 한다. 하지만 $3\times3$ 행렬 정도 라면 기호 연산도 용이하다.

In [ ]:
import sympy as sy

A = sy.Matrix(A)
A

In [ ]:
print('np.sum(A) = %r' % np.sum(A))
print('np.sum(A, 0) = %r' % np.sum(A, 0))
print('np.sum(A, 1) = %r' % np.sum(A, 1))
print('sy.det(A) = %r' % sy.det(A))
print('A.eigenvals() = %r' % A.eigenvals())
print('A.singular_values() = %r' % A.singular_values())

$4\times4$ 마방진은 르네상스 시대 알브레히트 뒤러의 동판화 *멜랑콜리아* 에 표현된 여러 수학적 오브제 가운데 하나이다.

In [ ]:
A = magic(4)
A

아래 명령들을 실행해 보면 각 행, 열, 대각선의 합이 모두 34로 A가 마방진임을 확인할 수 있다.

In [ ]:
print('np.sum(A, 0) = %r' % np.sum(A, 0))
print('np.sum(A, 1) = %r' % np.sum(A, 1))
print('np.sum(np.diag(A)) = %r' % np.sum(np.diag(A)))
print('np.sum(np.diag(np.flipud(A))) = %r' % np.sum(np.diag(np.flipud(A))))

그러나 이 $4\times4$ 마방진은 뒤러의 마방진과 같지는 않다. 두번째 세번째 열을 바꾸어야 한다.

In [ ]:
A = A[:, [0, 2, 1, 3]]
A

열 교환으로는 열 합계 또는 행 합계가 바뀌지 않는다.  보통 대각 합계는 바뀌지만 이경우에는 두 대각 합계 모두 34이다. 이제 이 마방진은 뒤러의 동판화와 일치하게 되었다. 뒤러가 이 행렬을 택한 이유는 아마도 작업했던 해인 1514년이 맨 아랫 줄에 나타났기 때문일 것이다.

지금까지 우리는 $4\times4$ 마방진 두개를 관찰했다. $4\times4$ 마방진은 모두 880개가 있고, $5\times5$ 마방진은 275305224가지가 있다는 점이 드러났다. $6\times6$ 이상의 마방진의 종류를 구분하는 것은 아직 해결되지 않은 문제이다.

우리의 $4\times4$ 마방진의 행렬식은 0이다. 만일 역행렬을 구하려고 한다면

In [ ]:
na.det(A)

In [ ]:
A.I

따라서 어떤 마방진은 특이행렬이라고 볼 수 있다.  어떤 행렬이 그러한가? 정방 행렬의 계수 rank 는 선형적으로 독립인 행 또는 열의 수 이다. $n\times n$ 행렬을 특이행렬이라고 볼 수 있는 필요 충분 조건은, 그 계수 rank 가 $n$ 보다 작은 것이다.

In [ ]:
r = np.array([(n, na.matrix_rank(magic(n))) for n in range(3, 24+1)])
r

위 결과를 조심스럽게 살펴 보자.  어떤 규칙성이 보이는가?  막대그래프에서 더 잘 보인다.

In [ ]:
import matplotlib.pyplot as plt


plt.bar(r[:, 0], r[:, 1])
plt.title('Rank of magic squares')
plt.show()

rank 에 따라 3가지 마방진이 있다.
* 홀수
* 홀수의 2배
* 짝수의 2배

홀수차 마방진은 full rank 이다. 이들은 특이행렬이 아니며 역행렬을 가진다. 짝수의 2배 차수 마방진은 n 이 아무리 크더라도 rank 는 3이다.  이들은 *매우 특이* 하다고 부를 수 있다.  홀수의 2배 차수 마방진의 rank 는 $n/2 + 2$이다.  이들도 특이행렬이지만, 행과 열이 짝수의 2배수 차수 보다 더 독립적이다.

마방진의 종류에 따라 3차원 곡면도 달라진다. 다양한 n 에 대해 다음을 시도해 보라.

In [ ]:
from mpl_toolkits.mplot3d.axes3d import Axes3D
import matplotlib.pyplot as plt

dummy = dir(Axes3D)

fig = plt.figure(figsize=(16,16))
fig.clf()

for n in range(8, 11+1):
    ax = fig.add_subplot(2, 2, n - 7, projection='3d')
    # help(ax)
    m = np.array(magic(n)) 
    x = np.arange(n)
    mx, my = np.meshgrid(x, x)
    ax.plot_surface(mx, my, m)
    plt.title('m%d' % n)
    plt.axis('off')
    plt.axis('equal')

plt.show()